In [1]:
import folium
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import random
import gams_magic

In [2]:
# read link data
road_file_path = os.path.join('..','road_file.csv')
road_file = pd.read_csv(road_file_path,names=["RDWY_LINK_ID","REF_SITE_FROM_ID","REF_SITE_TO_ID"])

# read road shape (shape file already converted using CRS in qgis)
shape_path = os.path.join("..","transportation_data","Middleton_Cross_Plains","Features","Middleton_Road_New.shp")
shape = shapefile.Reader(shape_path)

# read crash shape
crash_path = os.path.join("..","transportation_data","Middleton_Cross_Plains","Features","Crash_data_combined_2017_2020.shp")
crash = shapefile.Reader(crash_path)

# load reference point coordinate
reference_coordinate = None
reference_path = os.path.join("..","reference_coordinate")

with open(reference_path, 'rb') as f:
    reference_coordinate = pickle.load(f)

# Start Model

In [3]:
%reload_ext gams_magic
%gams_cleanup -k
%gams_reset

In [4]:
%%gams
$title turning network

option limcol = 0, limrow = 0, solprint = off;
set
    nodes
    roadID
    season
    seriousness
;

alias (nodes,i,j);
$gdxin ../turning_network_time_updated.gdx
$loadm nodes=dim1 nodes=dim2
parameter real_distance(nodes,nodes) distance of a road;
$load  real_distance=time
$gdxin


$gdxin ../turning_network_turn_updated.gdx
parameter turn(nodes,nodes) distance of a road;
$load  turn=turn
$gdxin

$gdxin ../crash_drisk0320.gdx
parameter risk(nodes,nodes) distance of a road;
$load  risk=drisk
$gdxin

$gdxin ../crash_count0320.gdx
parameter count(nodes,nodes) distance of a road;
$load  count=count
$gdxin

*display distance;


$gdxin ../turning_network_linkID_updated.gdx
set road(roadID<,i,j);
$load  road=road
$gdxin



$ontext
$gdxin ../crash_file.gdx
parameter  crash(nodes,nodes) number of crashes on a road;
$load  crash=crash
$gdxin
$offtext

$ontext
$gdxin ../crash_seriousness.gdx
$load season=dim3 seriousness=dim4
parameter crashS(nodes,nodes,season,seriousness) number of crashes on a road with different seasons and seriousness;
$load  crashS=serious_crash
$gdxin
$offtext

parameter distance(nodes,nodes);

distance(i,j) = real_distance(i,j);


set arc(nodes,nodes);

arc(i,j) = no;
arc(i,j)$(distance(i,j) > 0.5) = yes;
* adjust the distance
distance(i,j)$(arc(i,j)) = distance(i,j) -1;

scalar
    turn_const /100/
    intermediate_val /10/
;

parameter
    supply(nodes)
;

scalar origin,destination;
execseed = 1 + gmillisec(jnow);
origin = uniformint(1,card(nodes));
destination = uniformint(1,card(nodes));

*supply(nodes)$(ord(nodes) = origin) = 1;

*supply(nodes)$(ord(nodes) = destination) = -1;

supply('1666494_2') = 1;
supply('1662728_3') = -1;

free variable
    total_dist
;

integer variable
    flow(i,j)
;

flow.lo(i,j) = 0

equation
    balance(nodes)
    objective_shortestPath
    objective_safestPath
    objective_shortestPath2
    objective_short "shortest path formulation"
    intermediate_constr
    turn_constr
;


balance(i)..
    sum(arc(i,j), flow(i,j)) - sum(arc(j,i), flow(j,i)) =e= supply(i);
    
objective_short..
    total_dist =e= sum(arc(i,j),flow(i,j)*distance(i,j));
**********************************************************

intermediate_constr..
    sum(arc(i,j),flow(i,j)*risk(i,j)) =l= intermediate_val;
********************************************************** 

objective_shortestPath..
    total_dist =e= sum(arc(i,j),flow(i,j)*turn(i,j));

objective_shortestPath2..
    total_dist =e= sum(arc(i,j),flow(i,j)*distance(i,j) + 0*flow(i,j)*turn(i,j));
    


    
turn_constr..
    sum(arc(i,j),flow(i,j)*turn(i,j)) =l= turn_const;


model seasonalPath /balance, objective_shortestPath/;

model seasonalPath2 /balance, objective_shortestPath2, turn_constr/;

model shortest /balance, objective_short/;

model intermediate /balance, objective_short, intermediate_constr/;


***risk model

distance(i,j) = risk(i,j);

solve shortest using mip minimizing total_dist;

set roadChosenRisk(roadID);

scalar 
    roadChosenRisk_risk
;

roadChosenRisk(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosenRisk(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

roadChosenRisk_risk = sum(arc(i,j),flow.l(i,j)*risk(i,j));

***crash count model
distance(i,j) = count(i,j);

solve shortest using mip minimizing total_dist;

set roadChosenCount(roadID);

roadChosenCount(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosenCount(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

*** pure distance model
distance(i,j) = real_distance(i,j);

solve shortest using mip minimizing total_dist;

set roadChosenDistance(roadID);

scalar 
    roadChosenDistance_risk
;

roadChosenDistance(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosenDistance(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

roadChosenDistance_risk = sum(arc(i,j),flow.l(i,j)*risk(i,j));
*** turn model

distance(i,j) = turn(i,j);

solve shortest using mip minimizing total_dist;

set roadChosenTurn(roadID);

roadChosenTurn(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosenTurn(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;

*solve seasonalPath2 using mip minimizing total_dist;

*** intermediate model

distance(i,j) = real_distance(i,j);

intermediate_val = (0.2*roadChosenDistance_risk+1.8*roadChosenRisk_risk)/2;

solve intermediate using mip minimizing total_dist;

set roadChosenIntermediate(roadID);

roadChosenIntermediate(roadID)$( sum(road(roadID,i,j), flow.l(i,j)) > 0.5) = yes;
roadChosenIntermediate(roadID)$( sum(road(roadID,i,j), flow.l(j,i)) > 0.5) = yes;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1710.4478,3211,9043,MIP,CPLEX,0.171
1,Normal (1),Optimal Global (1),39.0000,3211,9043,MIP,CPLEX,0.153
2,Normal (1),Optimal Global (1),1565.0487,3211,9043,MIP,CPLEX,0.192
3,Normal (1),Optimal Global (1),9.0000,3211,9043,MIP,CPLEX,0.147
4,Normal (1),Optimal Global (1),1716.3144,3212,9043,MIP,CPLEX,0.63


In [5]:
'''
%gams_pull -d flow
%gams_pull supply
%gams_pull -d road
%gams_pull roadChosen
%gams_pull roadChosen2
%gams_pull roadChosen3
%gams_pull roadChosen_time
%gams_pull roadChosen_turn
%gams_pull roadChosen2_time
%gams_pull roadChosen2_turn
%gams_pull roadChosen3_time
%gams_pull roadChosen3_turn
'''

'\n%gams_pull -d flow\n%gams_pull supply\n%gams_pull -d road\n%gams_pull roadChosen\n%gams_pull roadChosen2\n%gams_pull roadChosen3\n%gams_pull roadChosen_time\n%gams_pull roadChosen_turn\n%gams_pull roadChosen2_time\n%gams_pull roadChosen2_turn\n%gams_pull roadChosen3_time\n%gams_pull roadChosen3_turn\n'

In [6]:
%gams_pull -d flow
%gams_pull supply
%gams_pull -d road
%gams_pull roadChosenRisk
%gams_pull roadChosenCount
%gams_pull roadChosenDistance
%gams_pull roadChosenTurn
%gams_pull roadChosenIntermediate

In [7]:
# get the chosen road
chosen_road_risk = []
chosen_road_count = []
chosen_road_distance = []
chosen_road_turn = []
chosen_road_intermediate = []
for link in roadChosenRisk:
    if "intersection" not in link:
        chosen_road_risk.append(int(link)) 
        
for link in roadChosenCount:
    if "intersection" not in link:
        chosen_road_count.append(int(link))
                                 
for link in roadChosenDistance:
    if "intersection" not in link:
        chosen_road_distance.append(int(link))
        
for link in roadChosenTurn:
    if "intersection" not in link:
        chosen_road_turn.append(int(link))
        
for link in roadChosenIntermediate:
    if "intersection" not in link:
        chosen_road_intermediate.append(int(link))
# get the chosen origin and destination
origin = None
destination = None
for i in supply:
    if i[1] == 1:
        origin = int(i[0].split("_")[0])
    else:
        destination = int(i[0].split("_")[0]) 

In [8]:
plt.rcParams["figure.figsize"] = (20,10)
plt.figure()

    
chosen_risk = []
chosen_count = []
chosen_distance = []
chosen_turn = []
chosen_intermediate = []
#plot original map 
for sp in shape.shapeRecords():
    road_id = sp.record[43]

    x = [i[0] for i in sp.shape.points[:]]
    y = [i[1] for i in sp.shape.points[:]]
        
    #record chosen road
    if road_id in chosen_road_risk:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_risk.append(segment)
    
    if road_id in chosen_road_count:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_count.append(segment)
        
    if road_id in chosen_road_distance:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_distance.append(segment)
        
    if road_id in chosen_road_turn:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_turn.append(segment)
        
    if road_id in chosen_road_intermediate:
        segment = []
        for i in range(len(x)):
                segment.append((y[i],x[i]))
        chosen_intermediate.append(segment)
        

<Figure size 1440x720 with 0 Axes>

In [9]:
m = folium.Map(location=[43.0819, -89.5579])

for loc in chosen_risk:
    folium.PolyLine(loc,
                color='red',
                weight=6,
                opacity=0.8).add_to(m)
'''   
for loc in chosen_count:
    folium.PolyLine(loc,
                color='blue',
                weight=5,
                opacity=0.8).add_to(m)
'''

for loc in chosen_distance:
    folium.PolyLine(loc,
                color='green',
                weight=4,
                opacity=0.8).add_to(m)
'''   
for loc in chosen_turn:
    folium.PolyLine(loc,
                color='orange',
                weight=3,
                opacity=0.8).add_to(m)
'''

for loc in chosen_intermediate:
    folium.PolyLine(loc,
                color='purple',
                weight=4,
                opacity=0.8).add_to(m)

In [10]:
m

In [371]:
#m.save("turn_vs_time_example2.html")